# Detectron2 Beginner's Tutorial

<img src="https://dl.fbaipublicfiles.com/detectron2/Detectron2-Logo-Horz.png" width="500">

Welcome to detectron2! This is the official colab tutorial of detectron2. Here, we will go through some basics usage of detectron2, including the following:
* Run inference on images or videos, with an existing detectron2 model
* Train a detectron2 model on a new dataset

You can make a copy of this tutorial by "File -> Open in playground mode" and make changes there. __DO NOT__ request access to this tutorial.


# Go to git repository

In [ ]:
from google.colab import drive
# drive.mount('/content/drive')

tdd_path = "/content/drive/MyDrive/Tufts Dental Database"
repo_path = "/content/drive/MyDrive/Github/tdd-model"
destination_path = f"{repo_path}/dataset"
# !cp -r "$tdd_path" "$destination_path"
%cd "$repo_path"
%ls

# Install detectron2

In [ ]:
import locale
import sys, os, distutils.core
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
import json

In [ ]:
%%time
%pip install pyyaml==5.1
# # # Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# # # See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
# # !git clone 'https://github.com/facebookresearch/detectron2'
# # dist = distutils.core.run_setup("./detectron2/setup.py")
# # !python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
# # sys.path.insert(0, os.path.abspath('./detectron2'))

# # # Properly install detectron2. (Please do not install twice in both ways)
# # %pip install 'git+https://github.com/facebookresearch/detectron2.git'

# # Or, to install it from a local clone:
# !git clone https://github.com/facebookresearch/detectron2.git
%pip install -e detectron2

In [ ]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

# Run a pre-trained detectron2 model

We first download an image from the COCO dataset:

In [ ]:
# !wget http://images.cocodataset.org/val2017/000000439715.jpg -q -O input.jpg
# im = cv2.imread("./input.jpg")
# cv2_imshow(im)

Then, we create a detectron2 config and a detectron2 `DefaultPredictor` to run inference on this image.

In [ ]:
# cfg = get_cfg()
# # add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
# cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
# cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# # Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
# predictor = DefaultPredictor(cfg)
# outputs = predictor(im)

In [ ]:
# # look at the outputs. See https://detectron2.readthedocs.io/tutorials/models.html#model-output-format for specification
# print(outputs["instances"].pred_classes)
# print(outputs["instances"].pred_boxes)

In [ ]:
# # We can use `Visualizer` to draw the predictions on the image.
# v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
# out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
# cv2_imshow(out.get_image()[:, :, ::-1])

# Train on a custom dataset

In this section, we show how to train an existing detectron2 model on a custom dataset in a new format.

We use [the balloon segmentation dataset](https://github.com/matterport/Mask_RCNN/tree/master/samples/balloon)
which only has one class: balloon.
We'll train a balloon segmentation model from an existing model pre-trained on COCO dataset, available in detectron2's model zoo.

Note that COCO dataset does not have the "balloon" category. We'll be able to recognize this new class in a few minutes.

## Prepare the dataset

Register the balloon dataset to detectron2, following the [detectron2 custom dataset tutorial](https://detectron2.readthedocs.io/tutorials/datasets.html).
Here, the dataset is in its custom format, therefore we write a function to parse it and prepare it into detectron2's standard format. User should write such a function when using a dataset in custom format. See the tutorial for more details.


In [ ]:
from detectron2.structures import BoxMode
from utils.dataset import get_abnormality_dicts, create_train_val_split, get_class_counts
import pickle

In [ ]:
%%time
repo_path = '/content/drive/MyDrive/Github/tdd-model/dataset'
train_path = os.path.join(repo_path, 'train.pkl')
val_path = os.path.join(repo_path, 'val.pkl')
if os.path.exists(train_path) and os.path.exists(val_path):
  with open(train_path, 'rb') as f:
    train = pickle.load(f)
  with open(val_path, 'rb') as f:
    val = pickle.load(f)
else:
  dataset_dicts = get_abnormality_dicts(repo_path, BoxMode.XYWH_ABS)
  train, val = create_train_val_split(dataset_dicts, train_path, val_path)

In [ ]:
# register dataset
# DatasetCatalog.remove('abnormalities_train')
# DatasetCatalog.remove('abnormalities_val')
DatasetCatalog.register('abnormalities_train', lambda: train)
DatasetCatalog.register('abnormalities_val', lambda: val)

MetadataCatalog.get('abnormalities').set(thing_classes=['benign_cyst_neoplasia', 'malignant_neoplasia', 'inflammation', 'dysplasia', 'metabolic/systemic', 'trauma', 'developmental'],
                                               thing_colors=[(255, 127, 80), (255, 0, 0), (255, 215, 0), (0, 255, 255), (128, 0, 128), (255, 160, 122), (100, 149, 237)])
abnormalities_metadata = MetadataCatalog.get("abnormalities")

To verify the dataset is in correct format, let's visualize the annotations of randomly selected samples in the training set:



In [ ]:
# dataset_dicts = get_balloon_dicts("balloon/train")
dataset_dicts = DatasetCatalog.get('abnormalities_train')
for d in random.sample(dataset_dicts, 3):
    print(d["file_name"][-8:])
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=abnormalities_metadata, scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    out_img = out.get_image()[:, :, ::-1]
    resized_img = cv2.resize(out_img, (0, 0), fx=0.5, fy=0.5)

    cv2_imshow(resized_img)

## Train!

Now, let's fine-tune a COCO-pretrained R50-FPN Mask R-CNN model on the balloon dataset. It takes ~2 minutes to train 300 iterations on a P100 GPU.


In [ ]:
import detectron2.data.transforms as T
from detectron2.data import DatasetMapper   # the default mapper
from detectron2.engine import DefaultTrainer
from detectron2.data import build_detection_train_loader, build_detection_test_loader

augmentation = [
    T.RandomFlip(horizontal=True, vertical=False, prob=0.5),
    T.RandomContrast(0.5, 1.5),
    T.RandomApply(tfm_or_aug=T.RandomCrop("absolute", (84, 161)),
                      prob=0.5)
]

class CustomTrainer(DefaultTrainer):
  @classmethod
  def build_train_loader(cls, cfg):
    return build_detection_train_loader(cfg, mapper=DatasetMapper(cfg, is_train=True, augmentations=augmentation))
  @classmethod
  def build_test_loader(cls, cfg, dataset_name):
    return build_detection_test_loader(cfg, dataset_name, mapper=DatasetMapper(cfg, False))

In [ ]:
%%time
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("abnormalities_train",)
cfg.DATASETS.TEST = ("abnormalities_val")
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml")  # Let training initialize from model zoo

cfg.SOLVER.IMS_PER_BATCH = 4  # 2, This is the real "batch size" commonly known to deep learning people
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 1000    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset, (1 epoch= total/batch size, 60 iter= 238/4)
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   # The "RoIHead batch size". 128 is faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 7  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
cfg.INPUT.MASK_FORMAT = "bitmask"
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = CustomTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

## Inference & evaluation using the trained model
Now, let's run inference with the trained model on the balloon validation dataset. First, let's create a predictor using the model we just trained:



In [ ]:
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.10   # confidence
cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.45 # iou
predictor = DefaultPredictor(cfg)

Then, we randomly select several samples to visualize the prediction results.

In [ ]:
from detectron2.utils.visualizer import ColorMode
dataset_dicts = DatasetCatalog.get('abnormalities_val')
with_abnormalities = [obj for obj in dataset_dicts if obj['annotations']]
# for d in random.sample(dataset_dicts, 3):
for d in with_abnormalities:
    name = d["file_name"][-8:]

    # prediction
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    v = Visualizer(im[:, :, ::-1],
                   metadata=abnormalities_metadata,
                   scale=0.5,
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    pred_out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    pred_out = pred_out.get_image()[:, :, ::-1]
    # cv2_imshow(pred_out.get_image()[:, :, ::-1])

    # label
    im = cv2.imread(d["file_name"])
    v = Visualizer(im[:, :, ::-1],
                            metadata=abnormalities_metadata,
                            scale=0.5)
    label_out = visualizer.draw_dataset_dict(d)
    label_out = label_out.get_image()[:, :, ::-1]
    # cv2_imshow(label_out.get_image()[:, :, ::-1])


    # plot prediction and label side-by-side
    fig, ax = plt.subplots(1, 2, figsize=(10, 10))
    ax[0].imshow(img1)
    ax[1].imshow(img2)
    plt.title(f'prediction vs label ({name})')
    plt.show()

In [ ]:
# for d in with_abnormalities:
#     print(d["file_name"][-8:])
#     img = cv2.imread(d["file_name"])
#     visualizer = Visualizer(img[:, :, ::-1], metadata=abnormalities_metadata, scale=0.5)
#     out = visualizer.draw_dataset_dict(d)
#     cv2_imshow(out.get_image()[:, :, ::-1])

We can also evaluate its performance using AP metric implemented in COCO API.
This gives an AP of ~70. Not bad!

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
# evaluator = COCOEvaluator("balloon_val", output_dir="./output")
# val_loader = build_detection_test_loader(cfg, "balloon_val")
# print(inference_on_dataset(predictor.model, val_loader, evaluator))

evaluator = COCOEvaluator("abnormalities_val", cfg, False, output_dir=cfg.OUTPUT_DIR)
val_loader = build_detection_test_loader(cfg, "abnormalities_val")
results = inference_on_dataset(trainer.model, val_loader, evaluator)

# another equivalent way to evaluate the model is to use `trainer.test`
val_loader = trai.build_test_loader(cfg, "abnormalities_val")
results = trainer.test(cfg, val_loader)